In [13]:
import json
import pandas as pd
import urllib.parse
from collections import Counter
import numpy as np

In [18]:
### PARAMETERS
# -------------

# jsonPath = "C:/Users/agrog/Documents/Oxford/analysis/effugium/studies/jan24Pilot/oxfordRoomworld24.json"
# csvPath = "C:/Users/agrog/Documents/Oxford/analysis/effugium/studies/jan24Pilot/roomworldPilotRounds.csv"
jsonPath = "20240205T1938_studies.json"
csvPath = "roomworldPilotRounds.csv"
# -------------

In [19]:
with open(jsonPath, 'rb') as h:
    qs = json.load(h)

data = [i['fields'] for i in qs]

In [20]:
dataDf = pd.DataFrame.from_dict(data)
df = dataDf.sort_values(by="lastModified", ascending=False)

In [23]:
df.iloc[0]["gameParameters"]["curriculumType"]

'strategy'

In [22]:
df = dataDf.loc[dataDf["UserId"].isin([
    "raddus",
    "testtesttest"
])]

In [24]:
def getRounds(df, save=False):
    # Expects a filtered dataframe from server json
    # Convert back to a dict to operate on
    data = pd.DataFrame.to_dict(df, orient="index")
    # Iterate over each participant
    roundLevelData = []
    for user in data:
        common = {'userId' : data[user]['UserId'], "lastModified" : data[user]["lastModified"], "timeCreated" : data[user]["timeCreated"], "debriefAnswers" : data[user]["riseTracking"], "studyId" : data[user]["gameParameters"]["study_id"], "sessionId" : data[user]["gameParameters"]["session_id"], "curriculumType" : data[user]["gameParameters"]["curriculumType"]}
        for attempt in data[user]['data']:
            tmp = {**common, **attempt}
            # The end game looping thing at the end was also spamming final round requests, so we need to remove these duplicates
            if len(roundLevelData) != 0 and roundLevelData[-1]["roundIndex"] == 94 and attempt["roundIndex"] == 94:
                pass
            else:
                roundLevelData.append(tmp)

    if save:
        rldf = pd.DataFrame.from_dict(roundLevelData)
        rldf.to_csv(csvPath, index=False)
    
    return roundLevelData

In [25]:
rld = getRounds(df, save=True)
# create dataframe
rldf = pd.DataFrame.from_dict(rld)

In [26]:
rldf.to_csv("roomworldPilotRounds.csv", index=False)

In [30]:
Counter(rldf["curriculumType"])

Counter({'strategy': 2384, 'concept': 2520})

In [28]:
rldf.query("userId == 'raddus' & roundType == 'transfer'")

,userId,lastModified,timeCreated,resp,level,layout,endTime,layoutId,completed,currLevel,roundType,startTime,attemptNum,fromReload,numCorrect,roundIndex,stratLevel,globalRoundNum,numCorrectPerLevel
85,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['t4', 't4', 'skip'], 'xloc': [9, 9, ...",3,", , ,-, , , ,-, , , ,-, , ,t1\n , , ,-, ,k6, ...",1706882490614,transfer_143,False,1,transfer,1706882490207,0,False,23,85,1,85,0
86,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [14, 14], '...",3,"p2, , ,-, , , ,-, , ,t7,-, , , \n ,k2, ,d2, ,k...",1706882491433,transfer_53,False,1,transfer,1706882491222,0,False,23,86,1,86,0
87,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [14, 0], 'y...",3,", , ,d12, , , ,-,x, ,t1,-, , , \n ,k12, ,-, ,...",1706882492352,transfer_14,False,1,transfer,1706882492049,0,False,23,87,1,87,0
88,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [13, 13], '...",3,", , ,-, , , ,d1, , , ,-, , , \n , , ,d10, , ,...",1706882493197,transfer_39,False,1,transfer,1706882492957,0,False,23,88,1,88,0
89,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [4, 4], 'yl...",3,", ,c8,-, , ,t8,-, , , ,-,c5, , \np5, , ,-, , ...",1706882495003,transfer_92,False,1,transfer,1706882493814,0,False,23,89,1,89,0
90,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [14, 14], '...",3,", , ,-, , , ,d6, , , ,-, , , \np9, , ,-,c2, ,...",1706882496059,transfer_110,False,1,transfer,1706882495610,0,False,23,90,1,90,0
91,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 'skip'], 'xloc': [5, 5], 'yl...",3,", ,p10,-, , ,c12,-, ,p10, ,-, ,t5, \n , , ,-,...",1706882496959,transfer_132,False,1,transfer,1706882496666,0,False,23,91,1,91,0
92,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['k8', 'skip'], 'xloc': [6, 10], 'ylo...",3,", , ,-, , , ,-, , ,i,-, , , \n , ,t7,-, , , ,...",1706882497849,transfer_74,False,1,transfer,1706882497567,0,False,23,92,1,92,0
93,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['t4', 't4', 'none', 'none', 'k9', 'k...",3,", , ,-, , , ,-, ,t2, ,-, , ,t5\n , , ,-,p12, ...",1706882503473,transfer_120,False,1,transfer,1706882498459,0,False,23,93,1,93,0
94,raddus,2024-02-02T14:09:05.763Z,2024-02-02T13:52:05.124Z,"{'tool': ['none', 't7', 't7', 'none', 't7', 't...",3,", ,p8,-, , , ,-, , , ,-, ,t8, \n , , ,d2,t11,...",1706882509345,transfer_91,False,1,transfer,1706882504079,0,False,23,94,1,94,0


In [29]:
rldf.query("userId == 'mtest'")

,userId,resp,level,layout,endTime,layoutId,completed,currLevel,roundType,startTime,attemptNum,fromReload,numCorrect,roundIndex,stratLevel,globalRoundNum,numCorrectPerLevel
125,mtest,"{'tool': ['none', 'k7', 'none', 'none', 'd7', ...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-, , , ,-...",1706868492364,level-c1-s1_49,True,1,train,1706868488280,0,False,1,0,1,0,1
126,mtest,"{'tool': ['none', 'skip'], 'xloc': [10, 10], '...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-, , , ,-...",1706868495850,level-c1-s1_4,False,1,train,1706868492976,0,False,1,1,1,1,0
127,mtest,"{'tool': ['k10', 'none', 'none', 'none', 'skip...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-, , , ,-...",1706868499105,level-c1-s1_31,False,1,train,1706868496461,0,False,1,2,1,2,0
128,mtest,"{'tool': ['none', 'none', 't10', 't10', 'none'...",1,", , ,d3, ,i, ,-, , , ,-,t1, , \n , , ,-,t2, ,...",1706868502615,level-c2-s1_45,True,1,train,1706868499716,0,False,2,3,1,3,1
129,mtest,"{'tool': ['t5', 't5', 'skip'], 'xloc': [6, 6, ...",1,", , ,-, , , ,-, , , ,-,t11, , \n , , ,-, , , ...",1706868505178,level-c2-s1_9,False,1,train,1706868503224,0,False,2,4,1,4,0
130,mtest,"{'tool': ['none', 'none', 't5', 't5', 'none', ...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-,t5,t1,k...",1706868510088,level-c2-s1_10,True,1,train,1706868505791,0,False,3,5,1,5,1
131,mtest,"{'tool': ['none', 'none', 'none', 'none', 'ski...",1,", , ,-,i,p4, ,-, , , ,-, , , \n , , ,-, ,p11,...",1706868513375,level-c3-s1_12,False,1,train,1706868510699,0,False,3,6,1,6,0
132,mtest,"{'tool': ['none', 'skip'], 'xloc': [2, 2], 'yl...",1,", , ,-, , , ,-, , , ,-,x, , \n , , ,-, , , ,-...",1706868515122,level-c3-s1_37,False,1,train,1706868513988,0,False,3,7,1,7,0
133,mtest,"{'tool': ['none', 'skip'], 'xloc': [9, 9], 'yl...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-, , , ,-...",1706868516700,level-c3-s1_23,False,1,train,1706868515733,0,False,3,8,1,8,0
134,mtest,"{'tool': ['none', 'skip'], 'xloc': [2, 2], 'yl...",1,", , ,-, , , ,-, , , ,-, , , \n , , ,-, , , ,-...",1706868517965,level-c4-s1_43,False,1,train,1706868517311,0,False,3,9,1,9,0


In [41]:
# Some summary statistics
accs = rldf.groupby(by="userId")["completed"].mean()
print(accs)
print(f"Mean accuracy: {np.mean(list(accs))}%")

userId
5e58fa19fc38d6104f798878    0.915789
5ee386d305dd19000acbe9eb    0.821053
Name: completed, dtype: float64
Mean accuracy: 0.868421052631579%
